In [2]:
import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pyplot as plt
import plotly.io as pio
#import squarify #treemap
import os
import matplotlib
import warnings



#to enable the inline plotting
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

sns.set_style("darkgrid")

In [3]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import classification_report

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import StandardScaler


from scipy.stats import normaltest

from pandas_profiling import ProfileReport

from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.style.palettes import PALETTES, SEQUENCES, color_palette

import lazypredict
from lazypredict.Supervised import LazyClassifier

warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# Function for EDA. Using the display() function to have  well-formatted tables. We are mainly using pandas to explore the datasets

def dataset_description(df_target):

    print('This is the Dataset shape: %s\n' % (df_target.shape, ))
    print('Dataset columns: %s\n' % df_target.columns)

    print('\nColumns description:\n')
    display(df_target.info())
    display(df_target.describe())  # describe the dataset

    print('\nNull values:\n')
    display(df_target.isnull().sum())  # Identify null values

#function performing a quick check on df_inspection to have best of pandas functions separated by a line
def quick_check(dataframe):
    print('First 5 rows %s\n')
    print(dataframe.head(2))
    print("=====================================")
    print('Dataframe shape %s\n')
    print(dataframe.shape)
    print("=====================================")
    print('Dataframe describe categorical %s\n')
    print(dataframe.describe(include=['O']))
    print("=====================================")
    print('Dataframe null values %s\n')
    print(dataframe.isnull().sum())
    print("=====================================")
    print('Dataframe value counts %s\n')
    print(dataframe.value_counts())
    print("=====================================")

#stats function
def stats(dataframe):
    print('Dataframe correlation %s\n')
    print(dataframe.corr())
    print("=====================================")
    print('Dataframe covariance %s\n')
    print(dataframe.cov())
    print("=====================================")
    print('Dataframe skew %s\n')
    print(dataframe.skew())
    print("=====================================")
    print('Dataframe kurtosis %s\n')
    print(dataframe.kurt())
    print("=====================================")

#create a function to normalize characters from a dataset's column in Spanish
def normalize_characters(df, column):
    df[column] = df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace('á', 'a')
    df[column] = df[column].str.replace('é', 'e')
    df[column] = df[column].str.replace('í', 'i')
    df[column] = df[column].str.replace('ó', 'o')
    df[column] = df[column].str.replace('ú', 'u')
    df[column] = df[column].str.replace('ñ', 'n')
    df[column] = df[column].str.replace('ü', 'u')
    df[column] = df[column].str.replace('ç', 'c')
    df[column] = df[column].str.replace('(', '')
    df[column] = df[column].str.replace(')', '')
    df[column] = df[column].str.replace('\'', '')
    df[column] = df[column].str.replace('´', '')
    df[column] = df[column].str.replace('`', '')
    df[column] = df[column].str.replace('’', '')
    return df.head(2)

#create function to change detypes in64 to int32 in a df
def change_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
    return df

In [5]:
#show all print outputs when using a function
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#display all columns
pd.set_option('display.max_columns', None)

In [6]:
#point to the folder where the data is stored for Pedro
os.chdir(r"C:\Users\alfon\Desktop\Máster IE - Business Analytics & Big Data\3rd Term\Sustainability Datathon\Data")

# Loading inspection data
df_inspection = pd.read_csv('inspections.csv')

# Loading network data
df_network = pd.read_csv('network.csv')

# Loading submission data
df_submission = pd.read_csv('sample_submission.csv')

# Loading train data
df_train = pd.read_csv('train_consolidated.csv')

In [7]:
df_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909733 entries, 0 to 909732
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   PipeId     909733 non-null  int64
 1   Incidence  909733 non-null  int64
dtypes: int64(2)
memory usage: 13.9 MB


In [8]:
#I want to know how many different PipeId are in df_submission
df_submission['PipeId'].nunique()

909733

There is just one inspection per each pipe in the period 2021 and 2022

In [9]:
sub_pipeline=pd.merge(df_submission, df_network, on='PipeId', how='left')


In [10]:
sub_pipeline.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 909733 entries, 0 to 909732
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   PipeId               909733 non-null  int64  
 1   Incidence            909733 non-null  int64  
 2   Province             909733 non-null  object 
 3   Town                 909733 non-null  object 
 4   YearBuilt            909733 non-null  int64  
 5   Material             909733 non-null  object 
 6   GasType              909733 non-null  object 
 7   Diameter             909733 non-null  float64
 8   Length               909733 non-null  float64
 9   Pressure             909733 non-null  float64
 10  NumConnections       909733 non-null  int64  
 11  NumConnectionsUnder  909733 non-null  int64  
 12  BoolBridle           909733 non-null  bool   
dtypes: bool(1), float64(3), int64(5), object(4)
memory usage: 91.1+ MB


In [11]:
#get the names of all the columns in df_trainç
df_train.columns

Index(['PipeId', 'InspectionYear', 'MonthsLastRev', 'Incidence', 'Province',
       'Town', 'YearBuilt', 'Diameter', 'Length', 'Pressure', 'NumConnections',
       'NumConnectionsUnder', 'BoolBridle', 'No_Inspections',
       'No_Incidences_Total', 'average_severity_pipe', 'relative_risk',
       'preventive_maintenance_rate', 'Probability_rate',
       'Average_MonthsLastRev', 'pipe_inspected_frequently',
       'Age_pipe_at_inspection', 'aspect', 'Relative_Thickness', 'pipe_area',
       'Total_Connections', 'area_connection', 'incidence_area',
       'connection_bool', 'Severity_high', 'Severity_medium', 'Severity_low',
       'gas_natural', 'Material_Acrylonitrile-Butadiene-Styrene',
       'Material_Copper', 'Material_Fiberglass-Reinforced Plastic',
       'Material_Polyethylene', 'Material_Polypropylene', 'Diameter2',
       'Length2', 'Pressure2', 'Average yearly temperature (°C)',
       'Min. Temperature (°C)', 'Max. Temperature (°C)',
       'Yearly Rainfall (mm)', 'Average y

In [13]:
#Converting diameter to int
sub_pipeline['Diameter'] = sub_pipeline['Diameter'].astype('int64')

In [14]:
#divide all values in Diameter by 1000 to convert to meters
sub_pipeline['Diameter'] = sub_pipeline['Diameter'] / 1000

In [15]:
#create a new variable called aspect that is equal to pressure divided by diameter multiplied by length
sub_pipeline['aspect']=(sub_pipeline['Pressure']/sub_pipeline['Diameter'])*sub_pipeline['Length']

In [16]:
#create a new column that divides the diameter by the pressure and name it Relative Thickness
sub_pipeline['Relative_Thickness'] = sub_pipeline['Diameter'] / sub_pipeline['Pressure']

In [17]:
#creating a column named pipe_area that multiplies diameter by lenght by pi

sub_pipeline['pipe_area'] = sub_pipeline['Diameter'] * sub_pipeline['Length'] * 3.1416

In [18]:
#Create a column called Total_Connections that adds NumConnections and NumConnectionsUnder
sub_pipeline['Total_Connections'] = sub_pipeline['NumConnections'] + sub_pipeline['NumConnectionsUnder']

In [19]:
#create a column named area_connection that divides pipe_area by NumConnections
sub_pipeline['area_connection'] = sub_pipeline['Total_Connections']/ sub_pipeline['pipe_area'] 

In [20]:
#create a boolean column named connection_bool that is 1 if NumConnections is greater than 1
sub_pipeline['connection_bool'] = np.where(sub_pipeline['NumConnections'] > 1, 1, 0)

In [21]:
# Converting Boolbride into  boolean variable
def boolbridle(x):
    return 1 if x == 'True' else 0

In [22]:
# Apply function on dataset
sub_pipeline['BoolBridle'] =sub_pipeline['BoolBridle'].apply(lambda x: boolbridle(x))

In [23]:
#hot encode GasType column in the train_copy dataframe subset
sub_pipeline = pd.get_dummies(sub_pipeline, columns=['GasType'], prefix = ['GasType'])

In [24]:
#delete GasType_Gas propano column
sub_pipeline = sub_pipeline.drop(['GasType_Gas propano'], axis=1)

In [25]:
#change name of GasType_Gas natural column to gas_natural
sub_pipeline = sub_pipeline.rename(columns={'GasType_Gas natural': 'gas_natural'})
sub_pipeline.head(1)

,PipeId,Incidence,Province,Town,YearBuilt,Material,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural
0,446859944,0,Valencia,Alginet,2012,PE,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1


In [26]:
sub_pipeline.head(1)

,PipeId,Incidence,Province,Town,YearBuilt,Material,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural
0,446859944,0,Valencia,Alginet,2012,PE,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1


In [27]:
print (sub_pipeline['Material'].unique())

['PE' 'PN' 'AO' 'FD' 'CU' 'ZD' 'FI' 'PA' 'FG']


In [28]:
#Map df_combined Material column to the names of materials
sub_pipeline['Material'] = sub_pipeline['Material'].map({'PE': 'Polyethylene', 'AO': 'Acrylonitrile-Butadiene-Styrene', 'FD': 'Fiberglass-Reinforced Plastic', 
    'FG': 'Fiberglass', 'PN': 'Polypropylene', 'PA': 'Polyamide', 'FO': 'Flexible Polyolefin', 'FI': 'Flexible Polyvinyl Chloride', 'CU': 'Copper', 
    'PV': 'Polyvinylidene Fluoride', 'ZD': 'Zinc-Coated Steel', 'ZA': 'Zinc-Aluminum', 'CP': 'Cast Iron', 'CS': 'Cast Steel', 
    'ZC': 'Zinc-Coated Steel', 'ZM': 'Zinc-Magnesium','ZN': 'Zinc', 'AL': 'Aluminum', 'ZP': 'Zinc-Coated Steel', 'ZF': 'Zinc-Aluminum-Magnesium'})

In [29]:
#Hot enconde Material column in the train_copy dataframe subset
sub_pipeline = pd.get_dummies(sub_pipeline, columns=['Material'])
sub_pipeline = sub_pipeline.drop(['Material_Fiberglass', 'Material_Zinc-Coated Steel', 'Material_Flexible Polyvinyl Chloride', 'Material_Polyamide'], axis=1)
sub_pipeline.head(1)

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene
0,446859944,0,Valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0


In [30]:
sub_pipeline.head(1)

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene
0,446859944,0,Valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0


In [31]:
from sklearn.preprocessing import PolynomialFeatures

In [32]:
#create polinomial features in train_copy for Diameter, Length, Pressure
poly = PolynomialFeatures(2)
sub_pipeline['Diameter2'] = poly.fit_transform(sub_pipeline[['Diameter']])[:,2]
sub_pipeline['Length2'] = poly.fit_transform(sub_pipeline[['Length']])[:,2]
sub_pipeline['Pressure2'] = poly.fit_transform(sub_pipeline[['Pressure']])[:,2]
sub_pipeline.head(1)


,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2
0,446859944,0,Valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0,0.01,1961.25,0.02


In [33]:
#point to the folder where the data is stored
os.chdir(r"C:\Users\alfon\Desktop\Máster IE - Business Analytics & Big Data\3rd Term\Sustainability Datathon\Data\Pedro")

# Loading provincias dataset
provincias = pd.read_excel('promedio_tiempo_provincia_anual.xlsx')

In [34]:
provincias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Province                         38 non-null     object 
 1   Average yearly temperature (°C)  38 non-null     float64
 2   Min. Temperature (°C)            38 non-null     float64
 3   Max. Temperature (°C)            38 non-null     float64
 4   Yearly Rainfall (mm)             38 non-null     int64  
 5   Average year Humidity (%)        38 non-null     float64
 6   Rainy days per year (days)       38 non-null     int64  
 7   Yearly Sun Hours (hours)         38 non-null     float64
dtypes: float64(5), int64(2), object(1)
memory usage: 2.5+ KB


In [35]:
normalize_characters(sub_pipeline, 'Province')
normalize_characters(provincias, 'Province')

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2
0,446859944,0,valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0,0.01,1961.25,0.02
1,428124500,0,valencia,Alginet,2012,0.16,110.72,0.15,2,0,0,103.80,1.07,55.65,2,0.04,1,1,0,0,0,1,0,0.03,12257.81,0.02


,Province,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours)
0,lleida,15.46,1.80,32.90,461,0.59,50,112.40
1,la rioja,12.38,1.70,26.90,633,0.72,87,85.20


In [36]:
#join train_copy and provincias on Province
sub_pipeline = sub_pipeline.merge(provincias, on='Province', how='left')

In [37]:
sub_pipeline

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours)
0,446859944,0,valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0,0.01,1961.25,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20
1,428124500,0,valencia,Alginet,2012,0.16,110.72,0.15,2,0,0,103.80,1.07,55.65,2,0.04,1,1,0,0,0,1,0,0.03,12257.81,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20
2,438428871,0,valencia,Alginet,2012,0.11,13.57,0.15,0,0,0,18.51,0.73,4.69,0,0.00,0,1,0,0,0,1,0,0.01,184.28,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20
3,429034569,0,valencia,Alginet,2012,0.11,327.69,0.15,3,0,0,446.85,0.73,113.24,3,0.03,1,1,0,0,0,1,0,0.01,107380.08,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20
4,411184477,0,valencia,Alginet,2007,0.20,932.76,0.15,0,0,0,699.57,1.33,586.07,0,0.00,0,1,0,0,0,1,0,0.04,870044.95,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,235426649,0,valencia,Paterna,2004,0.11,7.66,0.03,0,0,0,1.74,4.40,2.65,0,0.00,0,1,0,0,0,1,0,0.01,58.61,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20
909729,235426673,0,valencia,Paterna,2004,0.11,4.38,0.03,2,0,0,1.00,4.40,1.51,2,1.32,1,1,0,0,0,1,0,0.01,19.21,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20
909730,235426707,0,valencia,Paterna,2004,0.11,2.29,0.03,1,0,0,0.52,4.40,0.79,1,1.26,0,1,0,0,0,1,0,0.01,5.25,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20
909731,190956601,0,valencia,Paterna,2004,0.09,87.81,4.00,0,0,0,3902.62,0.02,24.83,0,0.00,0,1,0,0,0,1,0,0.01,7710.42,16.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20


In [41]:
#I want to order the dataframe grouping by pipeid and sorting by InspectionYear

df_train = df_train.sort_values(['PipeId', 'InspectionYear'], ascending=[True, True])

In [42]:
df_train

,PipeId,InspectionYear,MonthsLastRev,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,No_Inspections,No_Incidences_Total,average_severity_pipe,relative_risk,preventive_maintenance_rate,Probability_rate,Average_MonthsLastRev,pipe_inspected_frequently,Age_pipe_at_inspection,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,incidence_area,connection_bool,Severity_high,Severity_medium,Severity_low,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours)
0,489616,2013,16,0,ciudad real,Torralba de Calatrava,2012,0.06,7.79,0.40,0,0,0,1,0,4.00,0.00,0.00,0.00,21.40,1,1,49.47,0.16,1.54,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.00,60.72,0.16,15.44,1.50,33.70,438,0.55,52,107.50
1,489616,2015,22,0,ciudad real,Torralba de Calatrava,2012,0.06,7.79,0.40,0,0,0,2,0,4.00,0.00,0.00,0.00,21.40,1,3,49.47,0.16,1.54,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.00,60.72,0.16,15.44,1.50,33.70,438,0.55,52,107.50
2,489616,2016,22,0,ciudad real,Torralba de Calatrava,2012,0.06,7.79,0.40,0,0,0,3,0,4.00,0.00,0.00,0.00,21.40,1,4,49.47,0.16,1.54,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.00,60.72,0.16,15.44,1.50,33.70,438,0.55,52,107.50
3,489616,2018,23,0,ciudad real,Torralba de Calatrava,2012,0.06,7.79,0.40,0,0,0,4,0,4.00,0.00,0.00,0.00,21.40,1,6,49.47,0.16,1.54,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.00,60.72,0.16,15.44,1.50,33.70,438,0.55,52,107.50
4,489616,2020,24,0,ciudad real,Torralba de Calatrava,2012,0.06,7.79,0.40,0,0,0,5,0,4.00,0.00,0.00,0.00,21.40,1,8,49.47,0.16,1.54,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.00,60.72,0.16,15.44,1.50,33.70,438,0.55,52,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2804026,451995260,2019,24,0,lleida,Almacelles,2003,0.09,67.28,0.15,2,0,0,4,0,4.00,0.00,0.00,0.00,24.00,1,16,112.14,0.60,19.02,2,0.11,0.00,1,0,0,0,1,0,0,0,1,0,0.01,4527.14,0.02,15.46,1.80,32.90,461,0.59,50,112.40
2804027,451995309,2013,24,0,lleida,Almacelles,1998,0.20,92.22,0.15,0,0,0,1,0,4.00,0.00,0.00,0.00,24.00,1,15,69.16,1.33,57.94,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.04,8504.53,0.02,15.46,1.80,32.90,461,0.59,50,112.40
2804028,451995309,2015,24,0,lleida,Almacelles,1998,0.20,92.22,0.15,0,0,0,2,0,4.00,0.00,0.00,0.00,24.00,1,17,69.16,1.33,57.94,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.04,8504.53,0.02,15.46,1.80,32.90,461,0.59,50,112.40
2804029,451995309,2017,24,0,lleida,Almacelles,1998,0.20,92.22,0.15,0,0,0,3,0,4.00,0.00,0.00,0.00,24.00,1,19,69.16,1.33,57.94,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.04,8504.53,0.02,15.46,1.80,32.90,461,0.59,50,112.40


In [46]:
#Now I am fetching last value of each group

df_trainlast= df_train.groupby('PipeId').last()
df_trainlast

,InspectionYear,MonthsLastRev,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,No_Inspections,No_Incidences_Total,average_severity_pipe,relative_risk,preventive_maintenance_rate,Probability_rate,Average_MonthsLastRev,pipe_inspected_frequently,Age_pipe_at_inspection,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,incidence_area,connection_bool,Severity_high,Severity_medium,Severity_low,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours)
PipeId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
489616,2020,24,0,ciudad real,Torralba de Calatrava,2012,0.06,7.79,0.40,0,0,0,5,0,4.00,0.00,0.00,0.00,21.40,1,8,49.47,0.16,1.54,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.00,60.72,0.16,15.44,1.50,33.70,438,0.55,52,107.50
489645,2020,24,0,ciudad real,Torralba de Calatrava,2012,0.09,2.08,0.40,0,0,0,5,0,4.00,0.00,0.00,0.00,21.40,1,8,9.24,0.22,0.59,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.01,4.33,0.16,15.44,1.50,33.70,438,0.55,52,107.50
489646,2020,24,0,ciudad real,Torralba de Calatrava,2012,0.09,111.17,0.40,3,0,0,5,0,4.00,0.00,0.00,0.00,21.40,1,8,494.10,0.22,31.43,3,0.10,0.00,1,0,0,0,1,0,0,0,1,0,0.01,12359.44,0.16,15.44,1.50,33.70,438,0.55,52,107.50
489780,2020,24,0,toledo,Añover de Tajo,2009,0.11,7.49,5.00,0,0,0,5,0,4.00,0.00,0.00,0.00,23.00,1,11,340.23,0.02,2.59,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.01,56.03,25.00,15.46,1.80,33.40,453,0.57,52,108.40
489790,2020,24,0,toledo,Añover de Tajo,2012,0.11,4.44,4.00,0,0,0,5,0,4.00,0.00,0.00,0.00,19.80,1,8,161.31,0.03,1.53,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.01,19.68,16.00,15.46,1.80,33.40,453,0.57,52,108.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451195406,2017,24,0,malaga,Málaga,2012,0.16,0.82,0.15,0,0,0,3,0,4.00,0.00,0.00,0.00,20.00,1,5,0.77,1.07,0.41,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.03,0.68,0.02,17.60,8.90,28.00,485,0.70,48,110.40
451195430,2019,24,0,malaga,Málaga,2012,0.16,130.91,0.15,7,0,0,4,0,4.00,0.00,0.00,0.00,21.00,1,7,122.73,1.07,65.80,7,0.11,0.00,1,0,0,0,1,0,0,0,1,0,0.03,17138.48,0.02,17.60,8.90,28.00,485,0.70,48,110.40
451995254,2015,24,0,lleida,Almacelles,2010,0.09,0.09,0.15,0,0,0,2,0,4.00,0.00,0.00,0.00,34.67,0,5,0.16,0.60,0.03,0,0.00,0.00,0,0,0,0,1,0,0,0,1,0,0.01,0.01,0.02,15.46,1.80,32.90,461,0.59,50,112.40


In [60]:
#I want to add to the sub_pipeline dataframe the Inspection Year as 'LastInspectionYear' and the Average_MonthsLastRev from the df_trainlast dataframe just for the existing pipe_ids in the sub_pipeline datafram
sub_pipeline = sub_pipeline.merge(df_trainlast[['InspectionYear', 'Average_MonthsLastRev']], left_on='PipeId', right_index=True, how='left')



In [67]:
#Get all the columns of sub_pipeline
sub_pipeline.columns

Index(['PipeId', 'Incidence', 'Province', 'Town', 'YearBuilt', 'Diameter',
       'Length', 'Pressure', 'NumConnections', 'NumConnectionsUnder',
       'BoolBridle', 'aspect', 'Relative_Thickness', 'pipe_area',
       'Total_Connections', 'area_connection', 'connection_bool',
       'gas_natural', 'Material_Acrylonitrile-Butadiene-Styrene',
       'Material_Copper', 'Material_Fiberglass-Reinforced Plastic',
       'Material_Polyethylene', 'Material_Polypropylene', 'Diameter2',
       'Length2', 'Pressure2', 'Average yearly temperature (°C)',
       'Min. Temperature (°C)', 'Max. Temperature (°C)',
       'Yearly Rainfall (mm)', 'Average year Humidity (%)',
       'Rainy days per year (days)', 'Yearly Sun Hours (hours)',
       'InspectionYear', 'Average_MonthsLastRev'],
      dtype='object')

In [65]:
#drop columns InspectionYear_x', 'Average_MonthsLastRev_x', 'InspectionYear_y','Average_MonthsLastRev_y' Incidence from the sub_pipeline dataframe
sub_pipeline = sub_pipeline.drop(['InspectionYear_x', 'Average_MonthsLastRev_x', 'InspectionYear_y','Average_MonthsLastRev_y'], axis=1)

In [68]:
#I want to fill the null values of InspectionYear_y in the sub_pipeline dataframe with the value of Yearbuilt from df_network for the existing pipe ids in the sub_pipeline dataframe
sub_pipeline['InspectionYear'] = sub_pipeline['InspectionYear'].fillna(sub_pipeline['YearBuilt'])
#rename column InspectionYear_y to LastInspectionYear
sub_pipeline = sub_pipeline.rename(columns={'InspectionYear': 'LastInspectionYear'})

In [73]:
sub_pipeline

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours),LastInspectionYear,Average_MonthsLastRev,InspectionYear
0,446859944,0,valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0,0.01,1961.25,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,2019.00,20.75,2021.00
1,428124500,0,valencia,Alginet,2012,0.16,110.72,0.15,2,0,0,103.80,1.07,55.65,2,0.04,1,1,0,0,0,1,0,0.03,12257.81,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,2019.00,21.00,2021.00
2,438428871,0,valencia,Alginet,2012,0.11,13.57,0.15,0,0,0,18.51,0.73,4.69,0,0.00,0,1,0,0,0,1,0,0.01,184.28,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,2019.00,20.75,2021.00
3,429034569,0,valencia,Alginet,2012,0.11,327.69,0.15,3,0,0,446.85,0.73,113.24,3,0.03,1,1,0,0,0,1,0,0.01,107380.08,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,2019.00,20.75,2021.00
4,411184477,0,valencia,Alginet,2007,0.20,932.76,0.15,0,0,0,699.57,1.33,586.07,0,0.00,0,1,0,0,0,1,0,0.04,870044.95,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,2019.00,23.75,2021.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,235426649,0,valencia,Paterna,2004,0.11,7.66,0.03,0,0,0,1.74,4.40,2.65,0,0.00,0,1,0,0,0,1,0,0.01,58.61,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20,2020.00,24.00,2022.00
909729,235426673,0,valencia,Paterna,2004,0.11,4.38,0.03,2,0,0,1.00,4.40,1.51,2,1.32,1,1,0,0,0,1,0,0.01,19.21,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20,2020.00,24.00,2022.00
909730,235426707,0,valencia,Paterna,2004,0.11,2.29,0.03,1,0,0,0.52,4.40,0.79,1,1.26,0,1,0,0,0,1,0,0.01,5.25,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20,2020.00,24.00,2022.00
909731,190956601,0,valencia,Paterna,2004,0.09,87.81,4.00,0,0,0,3902.62,0.02,24.83,0,0.00,0,1,0,0,0,1,0,0.01,7710.42,16.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20,2020.00,24.00,2022.00


In [80]:
#Create a column adding 2 years to the LastInspectionYear column
sub_pipeline['InspectionYear'] = sub_pipeline['LastInspectionYear'] + 2

In [81]:
#Get all the values of the column InspectionYear which are not different than 2021 or 2022 and then set them to 2021
sub_pipeline['InspectionYear'] = np.where(sub_pipeline['InspectionYear'] != 2022, 2021, sub_pipeline['InspectionYear'])




In [83]:
sub_pipeline['InspectionYear'].value_counts()

2021.00    686123
2022.00    223610
Name: InspectionYear, dtype: int64

In [107]:
sub_pipeline

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours),Average_MonthsLastRev,InspectionYear,MonthsLastRev,No_Inspections
0,446859944,0,valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0,0.01,1961.25,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,20.75,2021.00,24.00,6.00
1,428124500,0,valencia,Alginet,2012,0.16,110.72,0.15,2,0,0,103.80,1.07,55.65,2,0.04,1,1,0,0,0,1,0,0.03,12257.81,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,21.00,2021.00,24.00,6.00
2,438428871,0,valencia,Alginet,2012,0.11,13.57,0.15,0,0,0,18.51,0.73,4.69,0,0.00,0,1,0,0,0,1,0,0.01,184.28,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,20.75,2021.00,24.00,6.00
3,429034569,0,valencia,Alginet,2012,0.11,327.69,0.15,3,0,0,446.85,0.73,113.24,3,0.03,1,1,0,0,0,1,0,0.01,107380.08,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,20.75,2021.00,24.00,6.00
4,411184477,0,valencia,Alginet,2007,0.20,932.76,0.15,0,0,0,699.57,1.33,586.07,0,0.00,0,1,0,0,0,1,0,0.04,870044.95,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,23.75,2021.00,24.00,6.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,235426649,0,valencia,Paterna,2004,0.11,7.66,0.03,0,0,0,1.74,4.40,2.65,0,0.00,0,1,0,0,0,1,0,0.01,58.61,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20,24.00,2022.00,24.00,7.00
909729,235426673,0,valencia,Paterna,2004,0.11,4.38,0.03,2,0,0,1.00,4.40,1.51,2,1.32,1,1,0,0,0,1,0,0.01,19.21,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20,24.00,2022.00,24.00,7.00
909730,235426707,0,valencia,Paterna,2004,0.11,2.29,0.03,1,0,0,0.52,4.40,0.79,1,1.26,0,1,0,0,0,1,0,0.01,5.25,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20,24.00,2022.00,24.00,7.00
909731,190956601,0,valencia,Paterna,2004,0.09,87.81,4.00,0,0,0,3902.62,0.02,24.83,0,0.00,0,1,0,0,0,1,0,0.01,7710.42,16.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20,24.00,2022.00,24.00,7.00


In [88]:
#get a new column named 'MonthsLastRev' which is the difference between the InspectionYear and the LastInspectionYear multiplied by 12
sub_pipeline['MonthsLastRev'] = (sub_pipeline['InspectionYear'] - sub_pipeline['LastInspectionYear']) * 12

In [91]:
#fill the null values of column average_months_last_rev with the value of MonthsLastRev
sub_pipeline['Average_MonthsLastRev'] = sub_pipeline['Average_MonthsLastRev'].fillna(sub_pipeline['MonthsLastRev'])

In [94]:
sub_pipeline.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 909733 entries, 0 to 909732
Data columns (total 37 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   PipeId                                    909733 non-null  int64  
 1   Incidence                                 909733 non-null  int64  
 2   Province                                  909733 non-null  object 
 3   Town                                      909733 non-null  object 
 4   YearBuilt                                 909733 non-null  int64  
 5   Diameter                                  909733 non-null  float64
 6   Length                                    909733 non-null  float64
 7   Pressure                                  909733 non-null  float64
 8   NumConnections                            909733 non-null  int64  
 9   NumConnectionsUnder                       909733 non-null  int64  
 10  BoolBridle          

In [96]:
#drop columns LastInspectionYear
sub_pipeline = sub_pipeline.drop(['LastInspectionYear'], axis=1)

KeyError: "['LastInspectionYear'] not found in axis"

In [99]:
#fetch the values in the column of No_inspections from the df_trainlast dataframe and add them to the sub_pipeline dataframe just for the existing pipe_ids in the sub_pipeline dataframe fill nan with 1
sub_pipeline = sub_pipeline.merge(df_trainlast['No_Inspections'], left_on='PipeId', right_index=True, how='left')

In [103]:

sub_pipeline = sub_pipeline.drop(['No_Inspections_x'], axis=1)

KeyError: "['No_Inspections_x'] not found in axis"

In [105]:
 #fill nan of No_Inspections_y with 0
sub_pipeline['No_Inspections_y'] = sub_pipeline['No_Inspections_y'].fillna(0)
#add 1 to all the values of No_Inspections_y
sub_pipeline['No_Inspections_y'] = sub_pipeline['No_Inspections_y'] + 1

In [106]:
#rename column No_Inspections_y to No_Inspections
sub_pipeline = sub_pipeline.rename(columns={'No_Inspections_y': 'No_Inspections'})

In [108]:
#fetch the values in the column of No_incidences_total from the df_trainlast dataframe and add them to the sub_pipeline dataframe just for the existing pipe_ids in the sub_pipeline dataframe fill nan with 0
sub_pipeline = sub_pipeline.merge(df_trainlast['No_Incidences_Total'], left_on='PipeId', right_index=True, how='left')

In [111]:
sub_pipeline.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 909733 entries, 0 to 909732
Data columns (total 38 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   PipeId                                    909733 non-null  int64  
 1   Incidence                                 909733 non-null  int64  
 2   Province                                  909733 non-null  object 
 3   Town                                      909733 non-null  object 
 4   YearBuilt                                 909733 non-null  int64  
 5   Diameter                                  909733 non-null  float64
 6   Length                                    909733 non-null  float64
 7   Pressure                                  909733 non-null  float64
 8   NumConnections                            909733 non-null  int64  
 9   NumConnectionsUnder                       909733 non-null  int64  
 10  BoolBridle          

In [113]:
#fill nan of No_Incidences_Total with 0
sub_pipeline['No_Incidences_Total'] = sub_pipeline['No_Incidences_Total'].fillna(0)

In [114]:
#Create in sub_pipeline a column called Age_pipe_at_inspection which is the difference between the InspectionYear and the YearBuilt
sub_pipeline['Age_pipe_at_inspection'] = sub_pipeline['InspectionYear'] - sub_pipeline['YearBuilt']

In [116]:
sub_pipeline

,PipeId,Incidence,Province,Town,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,aspect,Relative_Thickness,pipe_area,Total_Connections,area_connection,connection_bool,gas_natural,Material_Acrylonitrile-Butadiene-Styrene,Material_Copper,Material_Fiberglass-Reinforced Plastic,Material_Polyethylene,Material_Polypropylene,Diameter2,Length2,Pressure2,Average yearly temperature (°C),Min. Temperature (°C),Max. Temperature (°C),Yearly Rainfall (mm),Average year Humidity (%),Rainy days per year (days),Yearly Sun Hours (hours),Average_MonthsLastRev,InspectionYear,MonthsLastRev,No_Inspections,No_Incidences_Total,Age_pipe_at_inspection
0,446859944,0,valencia,Alginet,2012,0.11,44.29,0.15,0,0,0,60.39,0.73,15.30,0,0.00,0,1,0,0,0,1,0,0.01,1961.25,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,20.75,2021.00,24.00,6.00,0.00,9.00
1,428124500,0,valencia,Alginet,2012,0.16,110.72,0.15,2,0,0,103.80,1.07,55.65,2,0.04,1,1,0,0,0,1,0,0.03,12257.81,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,21.00,2021.00,24.00,6.00,0.00,9.00
2,438428871,0,valencia,Alginet,2012,0.11,13.57,0.15,0,0,0,18.51,0.73,4.69,0,0.00,0,1,0,0,0,1,0,0.01,184.28,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,20.75,2021.00,24.00,6.00,0.00,9.00
3,429034569,0,valencia,Alginet,2012,0.11,327.69,0.15,3,0,0,446.85,0.73,113.24,3,0.03,1,1,0,0,0,1,0,0.01,107380.08,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,20.75,2021.00,24.00,6.00,0.00,9.00
4,411184477,0,valencia,Alginet,2007,0.20,932.76,0.15,0,0,0,699.57,1.33,586.07,0,0.00,0,1,0,0,0,1,0,0.04,870044.95,0.02,17.62,6.60,30.80,427.00,0.64,46.00,113.20,23.75,2021.00,24.00,6.00,0.00,14.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,235426649,0,valencia,Paterna,2004,0.11,7.66,0.03,0,0,0,1.74,4.40,2.65,0,0.00,0,1,0,0,0,1,0,0.01,58.61,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20,24.00,2022.00,24.00,7.00,0.00,18.00
909729,235426673,0,valencia,Paterna,2004,0.11,4.38,0.03,2,0,0,1.00,4.40,1.51,2,1.32,1,1,0,0,0,1,0,0.01,19.21,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20,24.00,2022.00,24.00,7.00,0.00,18.00
909730,235426707,0,valencia,Paterna,2004,0.11,2.29,0.03,1,0,0,0.52,4.40,0.79,1,1.26,0,1,0,0,0,1,0,0.01,5.25,0.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20,24.00,2022.00,24.00,7.00,0.00,18.00
909731,190956601,0,valencia,Paterna,2004,0.09,87.81,4.00,0,0,0,3902.62,0.02,24.83,0,0.00,0,1,0,0,0,1,0,0.01,7710.42,16.00,17.62,6.60,30.80,427.00,0.64,46.00,113.20,24.00,2022.00,24.00,7.00,0.00,18.00


In [117]:
#Create a column called relative_risk which is the No_inspections multipled by nO_incidences_total
sub_pipeline['relative_risk'] = sub_pipeline['No_Inspections'] * sub_pipeline['No_Incidences_Total']

#Create a column called preventive_maintenance_rate which is the relative_risk divided by No_inspections
sub_pipeline['preventive_maintenance_rate'] = sub_pipeline['relative_risk'] / sub_pipeline['No_Inspections']

#Create a column called Probability_rate which is No_incidences_total divided by No_inspections
sub_pipeline['Probability_rate'] = sub_pipeline['No_Incidences_Total'] / sub_pipeline['No_Inspections']

In [ ]:
sub_pipeline.to_csv('submission_sample_prepared.csv')